# Left join in Spark



In [1]:
from pyspark.context import SparkContext, SparkConf
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F

conf=SparkConf()

sc = SparkContext.getOrCreate(conf=conf)
spark = SparkSession(sc)


In [2]:
import numpy as np
import pandas as pd

cats = ["a", "b", "c"]
defs = [f"The letter {e}" for e in cats]


num_rows = 1000
category = np.random.choice(cats,num_rows, p=[0.1, 0.3, 0.6])
values = np.random.randint(1, 100, num_rows)
pd_df_fact = pd.DataFrame({"cat": category, "val": values})

pd_df_dim = pd.DataFrame({"cat": cats, "def": defs})



In [3]:
df_fact = spark.createDataFrame(pd_df_fact)

In [4]:
df_dim = spark.createDataFrame(pd_df_dim)

In [5]:
df_fact.registerTempTable("df_fact")
df_dim.registerTempTable("df_dim")

sql = """
select df_fact.cat, df_dim.def, df_fact.val from
df_fact
left join df_dim
on df_fact.cat = df_dim.cat
"""

spark.sql(sql).write.csv("delete/", mode="overwrite")

In [5]:
## Can we re-write this in pyspark

val_sum = df.groupBy().agg(F.sum("val"))

df_perc = df.select((df["val"]/val_sum.collect()[0][0]).alias("perc"))
df_perc.write.csv("delete3/", mode="overwrite")


In [7]:
df.registerTempTable("df")

sql = """
select val/sum(val) over () as perc
from df

"""

spark.sql(sql).write.csv("delete2/", mode="overwrite")